# "Drop Number One" analysis

This is a modification of the "Drop One Variable" analysis that drops the top-ranked observation(s) instead of the series of input variables. This analysis tests the leverage of the "most vulnerable" observation on the net contributions of input variables to SoVI as well as the changes in index rank among US counties. If the index is robust to the leverage of individual observations, we expect to see minimal reshuffling of variable influence/county ranks when the top-ranked observation is dropped.

In [19]:
import os
import pandas as pd
import pysal as ps
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats.mstats import zscore as ZSCORE
from scipy.stats import rankdata
from scipy.stats import spearmanr 

# sovi compute script
import sys
sys.path.append(os.path.join(os.getcwd(),'code'))
from spss_pca import SPSS_PCA
import compute_sovis

## Methods

#### Some prep: 

In [20]:
#Get attribute names
attr_names=[j[0] for j in compute_sovis.input_names]

(Some of the stuff in the cell below might be removable. I just kept it all for now to make sure everything was in place for running SoVI...)

In [22]:
# rather than populating the `netContrib` object
# we can just use the final var contributions from 
# `compute_sovis`...
netContrib=compute_sovis.variable_contributions

#reorder table        
cols = ['USA', 'FEMA_1', 'g23g33g25', 
'FEMA_2', 'g36','FEMA_3', 'g51', 'FEMA_4', 'g13', 'FEMA_5', 'g17',
'FEMA_6', 'g48', 'FEMA_7', 'g29', 'FEMA_8', 'g46', 'FEMA_9', 'g06', 'FEMA_10', 
'g16']
netContrib = netContrib[cols]

#variable rank using absolute value      
rankContrib = abs(netContrib).apply(rankdata, axis=0, method='average')
rankContrib = (28-rankContrib) + 1

combContrib = pd.DataFrame(columns=list(netContrib.columns), index=list(netContrib.index))
#can't think of a more elegant way to do this
for aRow in range(netContrib.shape[1]):
    for aCol in range(netContrib.shape[0]):
        combContrib.ix[aCol][aRow] = str(round(netContrib.ix[aCol][aRow], 2)) + ' (' + str(int(rankContrib.ix[aCol][aRow])) + ')'

#build list of varIDs and human readable names
#sort and use as index for conContrib
nameSort = [[name, hrname] for name, sign, sample, hrname in compute_sovis.input_names]
nameSort = pd.DataFrame(nameSort)
nameSort.index = nameSort.loc[:,0]
nameSort = nameSort.reindex(list(combContrib.index))    
    
#set descriptive names
combContrib.index = list(nameSort.loc[:,1])

# # write out results
# combContrib

Get the index of the "most vulnerable" county in the USA according to SoVI (Bronx, NY), and generate a version of the SoVI inputs with it dropped:

In [23]:
# get the index of the 
# "most vulnerable" county in USA
# as a side note, it's Bronx, NY
sovi_no1=compute_sovis.US_Sovi_Score[compute_sovis.US_Sovi_Score['rank']==compute_sovis.US_Sovi_Score['rank'].min()].index

In [24]:
# the data without Bronx
US_drop_no1=compute_sovis.US_All.drop(sovi_no1)

Preserve the original net contribution and county ranks. These will be used to compute changes in net contribution/county rank once SoVI is recomputed:

In [25]:
# These are still needed for computing change in rank...

#Construct table to hold the results of the drop one analysis
#Sort variable list based on importance rank.
USvarRanks = rankContrib.USA.copy() #have to make a copy to sort index
USvarRanks.sort('USA')
# dropLevels = USvarRanks.index

C:\Users\Joe\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting


In [27]:
# preserve the original county ranks
# also for computing change in rank...
orig_rank=compute_sovis.US_Sovi_Score.drop(sovi_no1)['rank']

#### Now let's recompute SoVI with Bronx, NY dropped:

In [28]:
# preserve GEOIDs as an index
# for computed SoVI
geoLevels=US_drop_no1.Geo_FIPS

#Compute drop "number one"
pca = SPSS_PCA(US_drop_no1.drop(['Geo_FIPS', 'stateID'], axis = 1, inplace = False), reduce=True, varimax=True)
sovi_actual = pca.scores_rot.sum(1)
sovi_actual = pd.DataFrame(sovi_actual, index=geoLevels, columns=['sovi'])
US_SoVI_Drop1_Score = sovi_actual.values # SoVI score

In [29]:
# add SoVI ranks for run
# any way to clean this up?
US_SoVI_Drop1_Rank = pd.Series([i[0] for i in sovi_actual.values],index=geoLevels).rank(ascending=False)

In [30]:
# check indices match
sum(US_SoVI_Drop1_Rank.index==orig_rank.index)

3142

Recompute net variable contribution:

In [31]:
attrib_contribution = pd.Series(data=pca.weights_rot.sum(1), index=US_drop_no1.columns.drop(['Geo_FIPS', 'stateID']))
#print(j +" " + str(np.isnan(attrib_contribution.values).sum()))
attrib_contribution = attrib_contribution.transpose()

# attrib_contribution.index = [j]
#print(attrib_contribution.loc[j,:])
US_Drop1_NetContrib = attrib_contribution #.values
US_Drop1_NetContrib=US_Drop1_NetContrib.rank(ascending=False)
US_Drop1_NetContrib=US_Drop1_NetContrib.convert_objects(convert_numeric=True)
US_Drop1_NetContrib = US_Drop1_NetContrib.apply(lambda x: np.round(x, 2))

US_Drop1_NetContrib=US_Drop1_NetContrib[USvarRanks.index] # sort values by original index ranking

C:\Users\Joe\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


### Results

#### Net Contribution
Lots of small movements, and some larger ones for Percent African-American (`QBLACK_ACS`) and Percent of Housing in Mobile Homes (`QMOHO`), whose values drop considerably (-11 and -18, respectively).

In [16]:
nc_chg_dropno1=pd.DataFrame({'orig_rank':USvarRanks,'dropno1_rank':US_Drop1_NetContrib})
nc_chg_dropno1['rank_chg']=nc_chg_dropno1.orig_rank-nc_chg_dropno1.dropno1_rank
nc_chg_dropno1

dropno1_rank  orig_rank  rank_chg
QAGEDEP_ACS            1.0        1.0       0.0
QFEMALE_ACS            2.0        2.0       0.0
QSERV_ALT              3.0        3.0       0.0
QHISP_ACS              4.0        4.0       0.0
QFEMLBR                5.0        5.0       0.0
QNATAM_ACS             6.0        6.0       0.0
QESL_ALT               7.0        7.0       0.0
QSSBEN                 8.0        8.0       0.0
QNOAUTO_ALT            9.0        9.0       0.0
QMOHO                 28.0       10.0     -18.0
QPOVTY                10.0       11.0       1.0
QNRRES_ACS            11.0       12.0       1.0
QFAM                  12.0       13.0       1.0
QUNOCCHU_ACS          13.0       14.0       1.0
PERCAP_ALT            14.0       15.0       1.0
BLACK_ACS             27.0       16.0     -11.0
MDGRENT_ALT           15.0       17.0       2.0
QRICH200K             18.0       18.0       0.0
MEDAGE_ACS            16.0       19.0       3.0
QFHH_ACS              17.0       20.0       3.0
PRENTER_ACS           19.0       21.0       2.0
POPDENS               23.0       22.0      -1.0
QCVLUN                26.0       23.0      -3.0
MHSEVAL_ALT           20.0       24.0       4.0
QED12LES_ALT          21.0       25.0       4.0
QEXTRCT_ALT           22.0       26.0       4.0
QASIAN_ACS            25.0       27.0       2.0
QPUNIT_ACS            24.0       28.0       4.0

#### County Rank Change
At first glance, it looks like there could be a lot of movement in the county ranks when Bronx, NY is left out of the data. For example, a couple of counties in Alabama, `g01001` and `g01007`, shift from near the top of the index (578, and 326, respectively) to near the bottom (2839 and 2973, respectively).

In [17]:
obs_rchg_drop1=pd.DataFrame({'orig_rank':orig_rank,'dropno1_rank':US_SoVI_Drop1_Rank},index=orig_rank.index)
obs_rchg_drop1['rank_chg']=obs_rchg_drop1.orig_rank-obs_rchg_drop1.dropno1_rank
obs_rchg_drop1

dropno1_rank orig_rank rank_chg
Geo_FIPS                                 
g01001          2839.0       578    -2261
g01003          2183.0      1900     -283
g01005          1578.0      3124     1546
g01007          2973.0       326    -2647
g01009          2470.0      1303    -1167
g01011          2606.0       976    -1630
g01013           244.0       520      276
g01015          1649.0      3020     1371
g01017           704.0      1435      731
g01019          2387.0      1497     -890
g01021          2612.0       991    -1621
g01023          1945.0      2428      483
g01025          1013.0      2018     1005
g01027          1519.0      3022     1503
g01029          1943.0      2445      502
g01031          2020.0      2255      235
g01033          1904.0      2527      623
g01035           462.0       994      532
g01037          2217.0      1843     -374
g01039           804.0      1604      800
g01041          1846.0      2633      787
g01043          2022.0      2270      248
g01045          1955.0      2394      439
g01047           250.0       532      282
g01049          1549.0      3064     1515
g01051          2665.0       891    -1774
g01053          1266.0      2516     1250
g01055          1230.0      2436     1206
g01057          1524.0      3013     1489
g01059          1616.0      3090     1474
...                ...       ...      ...
g55129           540.0      1163      623
g55131          2944.0       386    -2558
g55133          3019.0       248    -2771
g55135          1673.0      2923     1250
g55137          1466.0      2925     1459
g55139          2403.0      1442     -961
g55141          1605.0      3078     1473
g56001          2981.0       306    -2675
g56003          2071.0      2136       65
g56005          3134.0        40    -3094
g56007          2769.0       677    -2092
g56009          2916.0       445    -2471
g56011          2878.0       499    -2379
g56013          1057.0      2122     1065
g56015          1944.0      2401      457
g56017           554.0      1177      623
g56019          2767.0       693    -2074
g56021          2480.0      1248    -1232
g56023          2933.0       403    -2530
g56025          2818.0       603    -2215
g56027           935.0      1836      901
g56029          2186.0      1885     -301
g56031           878.0      1770      892
g56033          2316.0      1641     -675
g56035          3135.0        31    -3104
g56037          3099.0       113    -2986
g56039          3032.0       228    -2804
g56041          2929.0       412    -2517
g56043          1777.0      2750      973
g56045          2873.0       508    -2365

[3142 rows x 3 columns]

In [18]:
sns.distplot(obs_rchg_drop1.rank_chg)

C:\Users\Joe\Anaconda3\lib\site-packages\statsmodels\nonparametric\kdetools.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  y = X[:m/2+1] + np.r_[0,X[m/2+1:],0]*1j
